In [150]:
import simpy
import numpy.random as random
import pandas as pd
import scipy.stats as st

from collections import namedtuple
from copy import deepcopy


def id_generator():
    """Generate unique, global customer ID."""
    i = 1
    while True:
        yield i
        i += 1
        
class IATDistribution:
    """
    Help control the table of Inter-Arrival Time Distribution table.
    """
    def __init__(self, path):
        self.df = pd.read_csv(path)
        self.df["Floor"] = self.df["Floor"].apply(lambda x:x.lstrip('0'))
        
    def getter(self, location, section, direction, floor):
        
        # filter row by mulitple conditions         
        temp = self.df[(self.df['Location'] == location) & (self.df['Section'] == section) & \
                       (self.df['Direction'] == direction) & (self.df['Floor'] == floor)][['Distribution','Parameters']]
        
        # return distribution name and parsed parameters
        return {
            'dist': getattr(st, temp['Distribution'].values[0]),
            'params': self.params_parser(temp['Parameters'].values[0])
        }
    
    @staticmethod
    def params_parser(params_string):
        params = []
        for param in params_string.replace("(", "").replace(")", "").split(", "): 
            params.append(float(param))
        return params

def displacement(floor1, floor2):
    floor1 = int(floor1) if not 'B' in floor1 else -int(floor1[1:])+1 
    floor2 = int(floor2) if not 'B' in floor2 else -int(floor2[1:])+1
    return floor2-floor1

class StopList:
    IDLE = 0
    ACTIVE = 1
    NA = -1
    
    def __init__(self, FloorList):
        self.floorlist = deepcopy(FloorList)
        num = displacement(FloorList[0], FloorList[len(FloorList)-1])
    
        self._list = {
             1: [0] * (num+1),
            -1: [0] * (num+1)
        }
        
        
        self.index = {
             1:{ floor:index for index, floor in enumerate(         FloorList ) },
            -1:{ floor:index for index, floor in enumerate(reversed(FloorList)) }
        }
        
        self.reversed_index = {
             1:{ index:floor for index, floor in enumerate(FloorList) },
            -1:{ index:floor for index, floor in enumerate(reversed(FloorList)) }
        }
        print(FloorList)
        print('Index: ',self.index)
        self.lowest = FloorList[0]
        self.highest = FloorList[len(FloorList)-1]
    
    def __str__(self):
        return str(self._list)
    
    def isEmpyty(self):
        for i, j in zip(self._list[1], self._list[-1]):
            if i == StopList.ACTIVE or j == StopList.ACTIVE:
                return False
        return True
    
    def pushOuter(self, elev, direction, floor):
        floor_index = self.index[direction][floor]
        if self._list[direction][floor_index] != StopList.NA:
            self._list[direction][floor_index] = StopList.ACTIVE
            return
        print('NANANANA')
        
    def pushInner(self, elev, destination):
        
        floor_index = self.index[elev.direction][destination]
        if self._list[elev.direction][floor_index] != StopList.NA:
            self._list[elev.direction][floor_index] = StopList.ACTIVE
            return
        print('NANANANA')
        
    
    def pop(self, elev):
        floor_index = self.index[elev.direction][elev.current_floor]
        if self._list[elev.direction][floor_index] != StopList.NA:
            self._list[elev.direction][floor_index] = StopList.IDLE
            return
        print('NANANANA')
    
    def next_target(self, elev):
        
        best_target = None
        peak = None
        
        
        # same direction
        curr_index = self.index[elev.direction][elev.current_floor]
        for i, state in enumerate(self._list[elev.direction][curr_index:]):
            if state == StopList.ACTIVE:
                best_target = self.reversed_index[elev.direction][i+curr_index]
                return best_target
            
            if state != StopList.NA:
                peak = self.reversed_index[elev.direction][i+curr_index]
                
        # Dirfferent direction
        for i, state in enumerate(self._list[-elev.direction]):
            if state == StopList.ACTIVE:
                return peak
        
        for i, state in enumerate(self._list[elev.direction][:curr_index]):
            if state == StopList.ACTIVE:
                best_target = self.reversed_index[elev.direction][i+curr_index]
                return best_target
        
        return None

In [165]:
Customer = namedtuple("Customer", ["cid", "destination"])
Mission = namedtuple("Mission", ["direction", "destination"])


class AssignmentError(Exception):
    """Exception : Fail to assign a elevator."""
    pass
    
class Queue:
    def __init__(self, env, floor, direction):
        self.env = env
        self.floor = floor
        self.direction = direction
        self.queue_array = []
        self.arrival_event = self.env.event()
        self.inflow_proc = self.env.process(self.inflow())
        self.outflow_proc = self.env.process(self.outflow())
        
    def inflow(self):
        while True:
            customers = yield self.arrival_event
            print('[INFLOW] Outer Call from {} Floor Going {} '.format(self.floor, 'up' if self.direction == 1 else 'down'))
            
            if(len(self.queue_array) == 0):
                global CALL_EVENT
                mission = Mission(direction=self.direction, destination=self.floor) ##
                CALL_EVENT.succeed(value=mission)   
                CALL_EVENT = self.env.event()
                
            self.queue_array = self.queue_array + customers  
            print('[INFLOW] {}'.format(self.queue_array))
                          
    def outflow(self):
        while True :
            capacity, occupationNum, elevIndex = yield AAA_EVENT[self.direction][self.floor]

            riders = []
            vacancy = capacity-occupationNum
            while (vacancy > 0) and (self.queue_array):
                riders.append(self.queue_array.pop())
                vacancy -= 1
            yield self.env.timeout(len(riders) * random.randint(WALKING_MIN, WALKING_MAX))
            
            print('[OUTFLOW] {} People Enters'.format(len(riders)))
                
            BBB_EVENT[elevIndex].succeed(value=riders)
            BBB_EVENT[elevIndex] = self.env.event()
    
class Floor:
    def __init__(self, env, floor, direction, IAT, DD):
        self.env = env
        self.floor = floor
        self.direction = direction
        self.queue = Queue(env, self.floor, self.direction)
        self.source_proc = env.process(self.Source(env))
        self.IAT = IAT
        self.DD = DD if len(DD) == 1 else DD/DD.sum()
        
    def Source(self, env):
        while True:
            
            # 1. set inter-arrival time based on given distribution
            t = -1
            while t < 0:
                t = self.IAT['dist'].rvs(*self.IAT['params'][:-2],loc = self.IAT['params'][-2],scale = self.IAT['params'][-1], size = 1)
            yield self.env.timeout(t)     

            # 2. set number of people of arrival group
            customers = []
            for i in range(20):#random.randint(1,20)
                
            # 3. set customer destination based on given posibility 
                to = random.choice(self.DD.index, p=self.DD)
                customers.append(Customer(cid=next(id_gen), destination=to))

            self.queue.arrival_event.succeed(value = customers)
            self.queue.arrival_event = self.env.event()


class Elevator:
    def __init__(self, env, elevIndex, failAssignment, floorList):
        self.env = env
        self.elevIndex = elevIndex
        self.current_floor = '1'
        self.capacity = 15
        self.riders = []
        
        self.stop_list = StopList(floorList)
        
        self.assign_event = self.env.event()
        self.direction = 0
        self.env.process(self.idle())
        self.failAssignment = failAssignment
            
    def idle(self):
        while True:
            print('[IDLE] Elevator {} Activated'.format(self.elevIndex))
            
            # first assignment
            mission = yield self.assign_event
            self.assign_event = self.env.event()
            
            print('[IDLE] Outer Call: {}'.format(mission))
            
            # push mission
            self.stop_list.pushOuter(self, mission.direction, mission.destination)

            # determine initial direction
            if displacement(self.current_floor, mission.destination) > 0:
                self.direction = 1
            elif displacement(self.current_floor, mission.destination) < 0:
                self.direction = -1
            else: 
                # the direction of the elevator and the mission may be the same floor
                self.direction = mission.direction    
            
            yield self.env.process(self.onMission())
            
            print('[IDLE] Elevator {} Stopped.'.format(self.elevIndex))
            self.direction = 0
#             self.resubmit()
    
    def onMission(self):
        while not self.stop_list.isEmpyty():
            
            print('[ONMISSION] On MISSION')
            print(self.stop_list)
            nextTarget = self.stop_list.next_target(self)
            print('[ONMISSION] NEXT TARGET {}'.format(nextTarget))
        
            moving_proc = self.env.process(self.moving( nextTarget, self.current_floor ))
            
            while self.current_floor != nextTarget:
                value = yield self.assign_event | moving_proc
                
                if self.assign_event.triggered:
                    mission = value[self.assign_event]
                    self.assign_event = self.env.event()
                    print('[ONMISSION] New Assigenment {}'.format(mission))
                    
                    before = nextTarget
                    self.stop_list.pushOuter(self, mission.direction, mission.destination)
                    
                    nextTarget = self.stop_list.next_target(self)
                    print('[ONMISSION] NEXT TARGET {}'.format(nextTarget))
                    if before != nextTarget:
                        
                        moving_proc.interrupt()
                        moving_proc = self.env.process(self.moving( nextTarget, self.current_floor ))
            
            print('[ONMISSION] Arrive At {} Floor'.format(self.current_floor))
            
            yield self.env.process(self.serving())
            print('[Afer Serving]',self.riders)
        
    def moving(self, destination, source):
        """source is needed to account for the acceleration and deceleration rate of the elevator"""
        print('[MOVING] Moving Process Started')
        try:   
            while self.current_floor != destination:
                
                # determine traveling time for 1 floor
                t = self.travelingTime(destination, self.current_floor, source)
                yield self.env.timeout(t)
                
                # advance 1 floor
                self.current_floor = self.forwards(self.current_floor, self.direction)
                
                print('[MOVING] Update To {} Floor'.format(self.current_floor))
        except simpy.Interrupt as i:
            print('[MOVING] Interrupted')
            print(self.stop_list)
            
        
    def serving(self):
     
        self.stop_list.pop(self)
        print('pop:',self.stop_list)
        
        # 放人
        countCus = 0
        for i in range(len(self.riders)-1, -1, -1):
            if self.riders[i].destination == self.current_floor:
                self.riders.pop(i)
                countCus += 1
        print('[SERVING] {} Customers Leave'.format(countCus))
        
        if not((self.current_floor == FloorList[-1] and self.direction == 1) or\
               (self.current_floor == FloorList[ 0] and self.direction == -1)):
            print('NOt PeAk')
            # boarding
            AAA_EVENT[self.direction][self.current_floor].succeed( value = (self.capacity, len(self.riders), self.elevIndex))
            AAA_EVENT[self.direction][self.current_floor] = self.env.event()  

            riders = yield BBB_EVENT[self.elevIndex]

            print('[SERVING] Customers Aboard: \n  ', riders)

            # new customers
            for customer in riders:
                self.stop_list.pushInner(self, customer.destination)
            print(self.stop_list)
            self.riders = self.riders + riders
        
        
        nextTarget = self.stop_list.next_target(self)
        print('[SERVING] NEXT TARGET {}'.format(nextTarget))
        if nextTarget is None:
            return
        else:
            if displacement(self.current_floor, nextTarget) > 0:
                print('GOING UP')
                self.direction = 1
            elif displacement(self.current_floor, nextTarget) < 0:
                print('GOING DOWN')
                self.direction = -1
            else:
                self.direction = -1*self.direction
                self.stop_list.pop(self)
                print('p!p:',self.stop_list)             
              
                # boarding
                AAA_EVENT[self.direction][self.current_floor].succeed( value = (self.capacity, len(self.riders), self.elevIndex))
                AAA_EVENT[self.direction][self.current_floor] = self.env.event()  

                riders = yield BBB_EVENT[self.elevIndex]

                print('[SERVING] Customers Aboard: \n  ', riders)

                # new customers
                for customer in riders:
                    self.stop_list.pushInner(self, customer.destination)
                print(self.stop_list)
                self.riders = self.riders + riders          
    
        
    def resubmit(self):
        for i in range(len(self.failAssignment)):
            mission = self.failAssignment.pop()
            global RESUBMIT_EVENT
            RESUBMIT_EVENT.succeed(value=mission)
            RESUBMIT_EVENT = self.env.event()
    
    def travelingTime(self, destination, current, source):
        return 10
    

    def boarding(self):
        AAA_EVENT[self.direction][self.current_floor].succeed( value = (self.capacity, len(self.riders), self.elevIndex))
        AAA_EVENT[self.direction][self.current_floor] = self.env.event()
        # Inner Calls
        riders = yield BBB_EVENT[self.elevIndex]
        print('[SERVING] Aboard: \n  ', riders)
        for customer in riders:
            self.stop_list.pushInner(self, customer.destination)
        print(self.stop_list)
        self.riders = self.riders + riders
        
    @staticmethod
    def mission_priority(mission):
        floor = mission.destination
        direction = mission.direction
        index = int(floor) if not 'B' in floor else -int(floor[1:]) + 1
        index = index*direction
        return index  
    
    @staticmethod
    def forwards(floor, direction):
        floor = int(floor) if not 'B' in floor else -int(floor[1:]) + 1
        floor += direction
        floor = str(floor) if floor > 0 else "B{}".format(abs(floor-1))
        return floor
        
class ElevatorController:
    def __init__(self, env, elevatorList, floorList):
        self.env = env
        self.elevatorList = elevatorList
        self.failAssignment = []
        self.elevators = dict()
        for elevatorName in elevatorList:
            self.elevators[elevatorName] = Elevator(env, elevatorName, self.failAssignment, floorList)
        self.env.process(self.assignCalls())
    
    def assignCalls(self):
        while True:
            mission = yield CALL_EVENT# | RESUBMIT_EVENT
#             for i in mission:
#                 mission = mission[i]
#             print('[{}] {}'.format('Call' if CALL_EVENT.triggered else 'Resubmit', mission))
            try:
#                 candidate = self.bestCandidate(mission.direction, mission.destination)
                candidate = self.elevators['3001'].elevIndex
        
                self.elevators[candidate].assign_event.succeed(value=mission)
                
            except AssignmentError:
                print('Fail to Assign')
                
    def bestCandidate(self, direction, source):
        """Assignment Policy"""
        minDistance = 50
        bestElevator = str()
        for elevator in self.elevators.values():
            if (minDistance > abs(displacement(source, elevator.current_floor)) and \
               ((direction == elevator.direction and elevator.current_floor*direction < direction*source) \
                or elevator.direction == 0)):
                    minDistance = abs(displacement(source, elevator.current_floor))
                    bestElevator = elevator.elevIndex
            
        if minDistance == 50:
            self.failAssignment.append(Mission(direction=direction, destination=source))
            raise AssignmentError()
            
        return bestElevator


In [166]:
random.seed(100)
    
# Inter-Arrival Time Distribution
IATD = IATDistribution('./BestFitDistribution - 複製.csv')

# Destination Distribution
DD = pd.read_csv('./FloorRatio_NHB.csv').iloc[:,1:].set_index('from').iloc[4:8,4:8]

# Location name
Location = "北棟病床"

# Time Section
TimeSection = 2

# Floor List
FloorList = ['1','2', '3', '4']

FloorDict = {}


# Elevator ID
ElevatorList = ['3001']


# Enviornment Variable
env = simpy.Environment()
id_gen = id_generator()
WALKING_MAX = 2
WALKING_MIN = 1

# Global Event
CALL_EVENT = env.event()
RESUBMIT_EVENT = env.event()
AAA_EVENT = { 1: { i : env.event() for i in FloorList}, -1: { i : env.event() for i in FloorList} }
BBB_EVENT = { i: env.event() for i in ElevatorList }

# process
floors_upward   = [ Floor(env, i,  1, IATD.getter(Location, TimeSection,   'up', i), DD.loc[i][i.lstrip("0"):]) for i in FloorList[ :-1]]
floors_downward = [ Floor(env, i, -1, IATD.getter(Location, TimeSection, 'down', i), DD.loc[i][:i.lstrip("0")]) for i in FloorList[1:  ]]
elevator_ctrl = ElevatorController(env, ElevatorList, FloorList)

env.run(until=10000)

['1', '2', '3', '4']
Index:  {1: {'1': 0, '2': 1, '3': 2, '4': 3}, -1: {'4': 0, '3': 1, '2': 2, '1': 3}}
[IDLE] Elevator 3001 Activated
[INFLOW] Outer Call from 1 Floor Going up 
[INFLOW] [Customer(cid=1, destination='4'), Customer(cid=2, destination='3'), Customer(cid=3, destination='3'), Customer(cid=4, destination='4'), Customer(cid=5, destination='3'), Customer(cid=6, destination='3'), Customer(cid=7, destination='2'), Customer(cid=8, destination='3'), Customer(cid=9, destination='4'), Customer(cid=10, destination='4'), Customer(cid=11, destination='3'), Customer(cid=12, destination='4'), Customer(cid=13, destination='3'), Customer(cid=14, destination='3'), Customer(cid=15, destination='4'), Customer(cid=16, destination='4'), Customer(cid=17, destination='3'), Customer(cid=18, destination='3'), Customer(cid=19, destination='3'), Customer(cid=20, destination='2')]
[IDLE] Outer Call: Mission(direction=1, destination='1')
[ONMISSION] On MISSION
{1: [1, 0, 0, 0], -1: [0, 0, 0, 0]}
[ONM

[INFLOW] Outer Call from 1 Floor Going up 
[INFLOW] [Customer(cid=1, destination='4'), Customer(cid=2, destination='3'), Customer(cid=3, destination='3'), Customer(cid=4, destination='4'), Customer(cid=5, destination='3'), Customer(cid=41, destination='3'), Customer(cid=42, destination='4'), Customer(cid=43, destination='4'), Customer(cid=44, destination='4'), Customer(cid=45, destination='3'), Customer(cid=46, destination='3'), Customer(cid=47, destination='3'), Customer(cid=48, destination='3'), Customer(cid=49, destination='3'), Customer(cid=50, destination='3'), Customer(cid=51, destination='2'), Customer(cid=52, destination='3'), Customer(cid=53, destination='3'), Customer(cid=54, destination='3'), Customer(cid=55, destination='3'), Customer(cid=56, destination='3'), Customer(cid=57, destination='4'), Customer(cid=58, destination='4'), Customer(cid=59, destination='3'), Customer(cid=60, destination='3'), Customer(cid=61, destination='3'), Customer(cid=62, destination='3'), Custome